In [ ]:
# version 1 #
import sys

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as canevas
from matplotlib.figure import Figure
import numpy as np

from PyQt5.QtCore import Qt, QPoint, QRect
from PyQt5.QtWidgets import QWidget, QApplication, QHBoxLayout, QVBoxLayout, QPushButton, QTableWidget, QTableWidgetItem, QLineEdit, QLabel

class fenetre(QWidget):
    """ la fenetre principale """
    def __init__(self):
        super().__init__()
        self.resize(700, 500)
        self.setWindowTitle('fitter v.1')
        
        # definition du canevas
        fig = Figure((4, 4))
        self.can = canevas(fig)
        self.can.setParent(self)
        self.graph = fig.add_subplot(111)
        
        # TableWidget
        self.tab_data=QTableWidget(100,3)
        self.tab_data.setHorizontalHeaderLabels(['x', 'y_data', 'y_fit'])
        
        # remplissage par une fonction sinus
        x = np.linspace(0,10,100)
        y_data = np.cos(x)
        y_fit = np.sin(x+0.1)
        self.fill_data(x, y_data)
        self.fill_fit(y_fit)
        self.plot()
        
        # lineEdit pour la formule
        self.le_formula = QLineEdit()
        self.le_formula.setText("np.sin(x+0.1)")
        lay_left = QVBoxLayout()
        lay_left.addWidget(self.le_formula)
        
        # appui sur entrée => mise à jour du graph correspondant à la formule
        entree = QShortcut('Return', self, self.update_fit)
        
        # ouverture du fichier
        btn_load = QPushButton("load data")
        btn_load.clicked.connect(self.load_data)
        self.lab_file = QLabel()
        lay_data = QHBoxLayout()
        lay_data.addWidget(btn_load)
        lay_data.addWidget(self.lab_file)
       
        # insertion du canevas dans un layout et affichage
        lay = QHBoxLayout()
        lay.addLayout(lay_left)
        lay.addWidget(self.can)
        lay.addWidget(self.tab_data)
        
        # top layout
        lay_top = QVBoxLayout(self)
        lay_top.addLayout(lay_data)
        lay_top.addLayout(lay)
        
        self.show()
    
    def fill_data(self, x, y):
        self.tab_data.setRowCount(x.shape[0])
        for i in range(x.shape[0]):
            self.tab_data.setItem(i, 0, QTableWidgetItem(str(x[i])))
            self.tab_data.setItem(i, 1, QTableWidgetItem(str(y[i])))
    
    def fill_fit(self, y):
        for i in range(y.shape[0]):
            self.tab_data.setItem(i, 2, QTableWidgetItem(str(y[i])))
            
    def plot(self):
        x = []
        y_data = []
        y_fit = []
        for i in range(self.tab_data.rowCount()):
            x.append(float(self.tab_data.item(i,0).text()))
            y_data.append(float(self.tab_data.item(i,1).text()))
            y_fit.append(float(self.tab_data.item(i,2).text()))
        self.graph.clear()
        self.graph.plot(x,y_data,'r')
        self.graph.plot(x,y_fit,'b')
        self.can.draw()
        
    def load_data(self):
        options = QFileDialog.Options()
        options |= QFileDialog.DontUseNativeDialog
        nom_fichier = QFileDialog.getOpenFileName(self, "load file :", options=options)
        self.lab_file.setText(nom_fichier[0])
        
        with open(nom_fichier[0], 'r') as file:
            data = file.readlines()
            x = np.zeros(len(data))
            y = np.zeros(len(data))
            for i,line in enumerate(data):
                xi,yi = line.split()
                x[i] = float(xi)
                y[i] = float(yi)
            self.fill_data(x,y)
            self.update_fit()   # on recalcule y_fit car les x peuvent avoir changé
        
    def update_fit(self):
        # récupération de x depuis la table des données (nécessaire pour l'évaluation de la formule)
        x = np.zeros(self.tab_data.rowCount())
        for i in range(self.tab_data.rowCount()):
            x[i] = float(self.tab_data.item(i,0).text())
            
        # calcul de y
        y = eval(self.le_formula.text())
        self.fill_fit(y)
        self.plot()
    
if __name__=='__main__':
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    f1 = fenetre()
    app.exec_()

In [ ]:
# version 2 #
import sys

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as canevas
from matplotlib.figure import Figure
import numpy as np

from PyQt5.QtCore import Qt, QPoint, QRect
from PyQt5.QtGui import QIcon
from PyQt5.QtWidgets import QWidget, QApplication, QHBoxLayout, QVBoxLayout, QPushButton, QTableWidget, QTableWidgetItem, QLineEdit, QLabel, QSpinBox, QShortcut

class fenetre(QWidget):
    """ la fenetre principale """
    def __init__(self):
        super().__init__()
        self.resize(700, 500)
        self.setWindowTitle('fitter v.2')
        
        # definition du canevas
        fig = Figure((4, 4))
        self.can = canevas(fig)
        self.can.setParent(self)
        self.graph = fig.add_subplot(111)
        
        # TableWidget
        self.tab_data=QTableWidget(100,3)
        self.tab_data.setHorizontalHeaderLabels(['x', 'y_data', 'y_fit'])
        
        # remplissage par une fonction sinus
        x = np.linspace(0,10,100)
        y_data = np.cos(x)
        y_fit = np.sin(x+0.1)
        self.fill_data(x, y_data)
        self.fill_fit(y_fit)
        self.plot()
        
        # lineEdit pour la formule + table de parametres
        self.le_formula = QLineEdit()
        self.le_formula.setStyleSheet("QLineEdit{color:blue;}")
        self.le_formula.setText("np.sin(x+0.1)+a")
        self.tab_param=QTableWidget(1,2)
        self.tab_param.setHorizontalHeaderLabels(['name', 'value'])
        self.tab_param.setItem(0, 0, QTableWidgetItem('a'))
        self.tab_param.setItem(0, 1, QTableWidgetItem('1'))
        self.spin = QSpinBox()
        self.spin.setValue(1)
        self.spin.valueChanged.connect(self.param_number_change)
        lay_param = QHBoxLayout()
        lay_param.addWidget(self.tab_param)
        lay_param.addWidget(self.spin)
        lay_left = QVBoxLayout()
        lay_left.addWidget(self.le_formula)
        lay_left.addLayout(lay_param)
        
        # appui sur entrée => mise à jour du graph correspondant à la formule
        entree = QShortcut('Return', self, self.update_fit)
        
        # ouverture du fichier
        btn_load = QPushButton("load data")
        btn_load.setShortcut('Alt+L')
        btn_load.clicked.connect(self.load_data)
        self.lab_file = QLabel()
        self.lab_file.setStyleSheet("QLabel {color : red;}")
        lay_data = QHBoxLayout()
        lay_data.addWidget(btn_load)
        lay_data.addWidget(self.lab_file)
       
        # insertion du canevas dans un layout et affichage
        lay = QHBoxLayout()
        lay.addLayout(lay_left)
        lay.addWidget(self.can)
        lay.addWidget(self.tab_data)
        
        # top layout
        lay_top = QVBoxLayout(self)
        lay_top.addLayout(lay_data)
        lay_top.addLayout(lay)
        
        self.show()
    
    def fill_data(self, x, y):
        self.tab_data.setRowCount(x.shape[0])
        for i in range(x.shape[0]):
            self.tab_data.setItem(i, 0, QTableWidgetItem(str(x[i])))
            self.tab_data.setItem(i, 1, QTableWidgetItem(str(y[i])))
    
    def fill_fit(self, y):
        for i in range(y.shape[0]):
            self.tab_data.setItem(i, 2, QTableWidgetItem(str(y[i])))
            
    def plot(self):
        x = []
        y_data = []
        y_fit = []
        for i in range(self.tab_data.rowCount()):
            x.append(float(self.tab_data.item(i,0).text()))
            y_data.append(float(self.tab_data.item(i,1).text()))
            y_fit.append(float(self.tab_data.item(i,2).text()))
        self.graph.clear()
        self.graph.plot(x,y_data,'r')
        self.graph.plot(x,y_fit,'b')
        self.can.draw()
        
    def load_data(self):
        options = QFileDialog.Options()
        options |= QFileDialog.DontUseNativeDialog
        nom_fichier = QFileDialog.getOpenFileName(self, "load file :", options=options)
        self.lab_file.setText(nom_fichier[0])
        
        with open(nom_fichier[0], 'r') as file:
            data = file.readlines()
            x = np.zeros(len(data))
            y = np.zeros(len(data))
            for i,line in enumerate(data):
                xi,yi = line.split()
                x[i] = float(xi)
                y[i] = float(yi)
            self.fill_data(x,y)
            self.update_fit()   # on recalcule y_fit car les x peuvent avoir changé
        
    def update_fit(self):
        # déclaration des parametre
        for i in range (self.tab_param.rowCount()):
            exec(self.tab_param.item(i,0).text() + '=' + self.tab_param.item(i,1).text())
           
        # récupération de x depuis la table des données (nécessaire pour l'évaluation de la formule)
        x = np.zeros(self.tab_data.rowCount())
        for i in range(self.tab_data.rowCount()):
            x[i] = float(self.tab_data.item(i,0).text())
        
        # calcul de y
        y = eval(self.le_formula.text())
        self.fill_fit(y)
        self.plot()
        
    def param_number_change(self):
        self.tab_param.setRowCount(self.spin.value())        
    
if __name__=='__main__':
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    f1 = fenetre()
    app.exec_()

In [1]:
# version 3 #
import sys

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as canevas
from matplotlib.figure import Figure
import numpy as np
from scipy.optimize import minimize

from PyQt5.QtCore import Qt, QPoint, QRect
from PyQt5.QtGui import QIcon
from PyQt5.QtWidgets import QWidget, QApplication, QHBoxLayout, QVBoxLayout, QPushButton, QTableWidget, QTableWidgetItem, QLineEdit, QLabel, QSpinBox, QShortcut

class fenetre(QWidget):
    """ la fenetre principale """
    def __init__(self):
        super().__init__()
        self.resize(700, 500)
        self.setWindowTitle('fitter v.3')
        
        # definition du canevas
        fig = Figure((4, 4))
        self.can = canevas(fig)
        self.can.setParent(self)
        self.graph = fig.add_subplot(111)
        
        # TableWidget
        self.tab_data=QTableWidget(100,3)
        self.tab_data.setHorizontalHeaderLabels(['x', 'y_data', 'y_fit'])
        
        # remplissage par une fonction sinus
        x = np.linspace(0,10,100)
        y_data = np.cos(x)
        y_fit = np.sin(x+0.1)
        self.fill_data(x, y_data)
        self.fill_fit(y_fit)
        self.plot()
        
        # lineEdit pour la formule + table de parametres
        self.le_formula = QLineEdit()
        self.le_formula.setStyleSheet("QLineEdit{color:blue;}")
        self.le_formula.setText("np.sin(x+0.1)+a")
        self.tab_param=QTableWidget(1,3)
        self.tab_param.setHorizontalHeaderLabels(['name', 'value', 'fit'])
        self.tab_param.setItem(0, 0, QTableWidgetItem('a'))
        self.tab_param.setItem(0, 1, QTableWidgetItem('1'))
        self.tab_param.setItem(0, 2, QTableWidgetItem(''))
        self.tab_param.item(0,2).setFlags(self.tab_param.item(0,2).flags() ^ ~Qt.ItemIsEnabled)
        self.spin = QSpinBox()
        self.spin.setValue(1)
        self.spin.valueChanged.connect(self.param_number_change)
        lay_param = QHBoxLayout()
        lay_param.addWidget(self.tab_param)
        lay_param.addWidget(self.spin)
        lay_left = QVBoxLayout()
        lay_left.addWidget(self.le_formula)
        lay_left.addLayout(lay_param)
        
        # bouton fit
        btn_fit = QPushButton("Launch Fit")
        btn_fit.clicked.connect(self.fit_data)
        lay_left.addWidget(btn_fit)
        
        # appui sur entrée => mise à jour du graph correspondant à la formule
        entree = QShortcut('Return', self, self.update_fit)
        
        # ouverture du fichier
        btn_load = QPushButton("load data")
        btn_load.setShortcut('Alt+L')
        btn_load.clicked.connect(self.load_data)
        self.lab_file = QLabel()
        self.lab_file.setStyleSheet("QLabel {color : red;}")
        lay_data = QHBoxLayout()
        lay_data.addWidget(btn_load)
        lay_data.addWidget(self.lab_file)
       
        # insertion du canevas dans un layout et affichage
        lay = QHBoxLayout()
        lay.addLayout(lay_left)
        lay.addWidget(self.can)
        lay.addWidget(self.tab_data)
        
        # top layout
        lay_top = QVBoxLayout(self)
        lay_top.addLayout(lay_data)
        lay_top.addLayout(lay)
        
        self.show()
    
    def fill_data(self, x, y):
        self.tab_data.setRowCount(x.shape[0])
        for i in range(x.shape[0]):
            self.tab_data.setItem(i, 0, QTableWidgetItem(str(x[i])))
            self.tab_data.setItem(i, 1, QTableWidgetItem(str(y[i])))
    
    def fill_fit(self, y):
        for i in range(y.shape[0]):
            self.tab_data.setItem(i, 2, QTableWidgetItem(str(y[i])))
            
    def plot(self):
        x = []
        y_data = []
        y_fit = []
        for i in range(self.tab_data.rowCount()):
            x.append(float(self.tab_data.item(i,0).text()))
            y_data.append(float(self.tab_data.item(i,1).text()))
            y_fit.append(float(self.tab_data.item(i,2).text()))
        self.graph.clear()
        self.graph.plot(x,y_data,'r')
        self.graph.plot(x,y_fit,'b')
        self.can.draw()
        
    def load_data(self):
        options = QFileDialog.Options()
        options |= QFileDialog.DontUseNativeDialog
        nom_fichier = QFileDialog.getOpenFileName(self, "load file :", options=options)
        self.lab_file.setText(nom_fichier[0])
        
        with open(nom_fichier[0], 'r') as file:
            data = file.readlines()
            x = np.zeros(len(data))
            y = np.zeros(len(data))
            for i,line in enumerate(data):
                xi,yi = line.split()
                x[i] = float(xi)
                y[i] = float(yi)
            self.fill_data(x,y)
            self.update_fit()   # on recalcule y_fit car les x peuvent avoir changé
        
    def update_fit(self, fitted=False):
        # déclaration des parametres
        if fitted:
            for i in range (self.tab_param.rowCount()):
                exec(self.tab_param.item(i,0).text() + '=' + self.tab_param.item(i,2).text())
        else:
            for i in range (self.tab_param.rowCount()):
                exec(self.tab_param.item(i,0).text() + '=' + self.tab_param.item(i,1).text())
           
        # récupération de x depuis la table des données (nécessaire pour l'évaluation de la formule)
        x = np.zeros(self.tab_data.rowCount())
        for i in range(self.tab_data.rowCount()):
            x[i] = float(self.tab_data.item(i,0).text())
        
        # calcul de y
        y = eval(self.le_formula.text())
        self.fill_fit(y)
        self.plot()
        
    def param_number_change(self):
        self.tab_param.setRowCount(self.spin.value())
        for i in range(self.tab_param.rowCount()):
            it = self.tab_param.item(i,2)
            if self.tab_param.item(i,2) is None:
                self.tab_param.setItem(i, 2, QTableWidgetItem(''))    
                self.tab_param.item(i,2).setFlags(self.tab_param.item(i,2).flags() ^ ~Qt.ItemIsEnabled)
    
    def error_square_sum(self, x):
        # initialisation des paramètres avec le vecteur x
        for i in range(self.tab_param.rowCount()):
            exec(self.tab_param.item(i,0).text() + '=' + str(x[i]))
            
        # récupération de x et y_data depuis la table des données 
        x = np.zeros(self.tab_data.rowCount())
        y_data = np.zeros(self.tab_data.rowCount())
        for i in range(self.tab_data.rowCount()):
            x[i] = float(self.tab_data.item(i,0).text())
            y_data[i] = float(self.tab_data.item(i,1).text())
        
        # calcul de y
        y_fit = eval(self.le_formula.text())
        
        # calcul de l'erreur
        return np.sum(np.square(y_data-y_fit))
        
    
    def fit_data(self):
        # création du vecteur de paramètres initiaux
        params = []
        for i in range(self.tab_param.rowCount()):
            params.append(float(self.tab_param.item(i,1).text()))
        initial_parameters = np.asarray(params)
        
        # procédure de fit
        result = minimize(self.error_square_sum, initial_parameters, method='nelder-mead')
        
        # mise à jour de la table et des courbes
        for i in range(self.tab_param.rowCount()):
            self.tab_param.item(i,2).setText(str(result.x[i]))
        self.update_fit(True)
        
if __name__=='__main__':
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    f1 = fenetre()
    app.exec_()